## Aluno: José Maria Clementino Junior NUSP: 11357281

- Link Repositório: https://github.com/clementinojr/SCC5836-Visualiza-o-Computacional
- Uma etapa prévia aos procedimentos realizados é a conversão das imagens no formato DICOM para PNG 
    - O código pode ser encontrado no arquivo "convertIMGtoPND.ipynb" no diretório de envio, como também no repositório do GitHub.

#### Descrição das bibliotecas utilizadas
  - Bibliotecas utilizadas para o tratamento de imagens:
      - cv2 : OpenCv
      - PIL 
   - Bibliotecas utilizadas para a realizar a estração de caracteristicas das imagens:
      - glob
      - mahotas
      - from __future__ import division
      - convolve2d
   - Bibliotecas utilizadas para a manipulação de arquivos:
      - os
      - csv
   - Bibliotecas utilizadas para a controle de tempo:
      - time
      
   - Bibliotecas utilizadas para a manipulação e visualização dos dados:
      - pandas
      - matplotlib
      - numpy
      

In [1]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from PIL import Image 
import mahotas 
import matplotlib.pylab as plt
import csv
import time
start_time = time.time()
from __future__ import division # Utilizado para LPQ ex
from scipy.signal import convolve2d # # Utilizado para LPQ ex

## Na sequeência são apresentados os 3 extratores de caracteristicas de textura, que são computados pelas 3 funções: 
    - fos : FOS - First Order Statistica
    - glcm_features  - Gray Level Co-ocurrence Level
    - lpq_features -  Local Phase Quantization

### def fos_features :
Funcão responsável por realizar a extração de caracteristica -Extrator FOS 

- Entrada:
  - uma imagem N x N  e uma mascara 

- Saida:
   - As sequintes features: features: 1)Mean, 2)Variance, 3)Median (50-Percentile), 4)Mode, 
                5)Skewness, 6)Kurtosis, 7)Energy, 8)Entropy, 
                9)Minimal Gray Level, 10)Maximal Gray Level, 
                11)Coefficient of Variation, 12,13,14,15)10,25,75,90-
                Percentile, 16)Histogram width
                

In [2]:
# -*- coding: utf-8 -*-

def fos_features(f, mask):
    
    # 1) Labels
    labels = ["FOS_Mean","FOS_Variance","FOS_Median","FOS_Mode","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth"]
    
    # 2) Parametros
    f  = f.astype(np.uint8)
    mask = mask.astype(np.uint8)
    level_min = 0
    level_max = 255
    Ng = (level_max - level_min) + 1
    bins = Ng
    
    # 3) Calculo do Histogram H sobre o ROI
    f_ravel = f.ravel() 
    mask_ravel = mask.ravel() 
    roi = f_ravel[mask_ravel.astype(bool)] 
    H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]
    
    # 4) Calculo das  Features
    features = np.zeros(16,np.double)  
    i = np.arange(0,bins)
    features[0] = np.dot(i,H)
    features[1] = sum(np.multiply(((i-features[0])**2),H))
    features[2] = np.percentile(roi,50) 
    features[3] = np.argmax(H)
    features[4] = sum(np.multiply(((i-features[0])**3),H))/(np.sqrt(features[1])**3)
    features[5] = sum(np.multiply(((i-features[0])**4),H))/(np.sqrt(features[1])**4)
    features[6] = sum(np.multiply(H,H))
    features[7] = -sum(np.multiply(H,np.log(H+1e-16)))
    features[8] = min(roi)
    features[9] = max(roi)
    features[10] = np.sqrt(features[2]) / features[0]
    features[11] = np.percentile(roi,10) 
    features[12] = np.percentile(roi,25)  
    features[13] = np.percentile(roi,75) 
    features[14] = np.percentile(roi,90) 
    features[15] = features[14] - features[11]
    
    return features, labels


### def glcm_features :
Funcão responsável por realizar a extração de caracteristica - Extrator GLCM 
- Entrada:
    - f: imagem das dimensões N1 x N2
    - d: distância para calcular a matriz de co-ocorrência (padrão d = 1)
    - th: ângulo para calcular a matriz de co-ocorrência (padrão th = [0,45,90,135])
    - ignore_zeros: ignorar zeros devido à máscara (padrão True)
- Outputs:
    - features:      Haralick's 1)Angular Second Moment, 2)Contrast, 
                     3)Correlation, 4)Sum of Squares: Variance, 5)Inverse 
                     Difference Moment 6)Sum Average, 7)Sum Variance, 8)Sum 
                     Entropy, 9)Entropy, 10)Difference Variance, 11)Difference 
                     Entropy, 12)Information Measure of Correlation 1, 
                     13)Information Measure of Correlation 2, 14)Maximal 
                     Correlation Coefficient    


In [3]:
# -*- coding: utf-8 -*-
def glcm_features(f, ignore_zeros=True):
    
    # 1) Labels
    labels = ["GLCM_ASM", "GLCM_Contrast", "GLCM_Correlation",
              "GLCM_SumOfSquaresVariance", "GLCM_InverseDifferenceMoment",
               "GLCM_SumAverage", "GLCM_SumVariance", "GLCM_SumEntropy",
               "GLCM_Entropy", "GLCM_DifferenceVariance",
               "GLCM_DifferenceEntropy", "GLCM_Information1",
               "GLCM_Information2", "GLCM_MaximalCorrelationCoefficient"]
    labels_mean = [label + "_Mean" for label in labels]
    labels_range = [label + "_Range" for label in labels]
    
    # 2) Parameters
    f = f.astype(np.uint8)
    
    # 3) Calculate Features: Mean and Range
    features = mahotas.features.haralick(f, 
                                         ignore_zeros=True, 
                                         compute_14th_feature=True,
                                         return_mean_ptp=True)
    features_mean = features[0:14]
    features_range = features[14:]
    
    return features_mean, features_range, labels_mean, labels_range


### def lpq_features :
Funcão responsável por realizar a extração de caracteristica - Extrator LPQ 
- Entrada:
    - img: imagem das dimensões N x N
    - winSize: tamanho da janela 
    - freqestim: Janela uniforme STFT
    - mode: definição do tipo de histograma 
- Outputs:
    - features: Histogramas das janelas
    

In [4]:
def lpq_features(img,winSize=3,freqestim=1,mode='nh'):
    rho=0.90

    STFTalpha=1/winSize  # alfa em abordagens STFT (para derivada gaussiana alfa = 1)
    sigmaS=(winSize-1)/4 # Sigma para janela STFT Gaussian (aplicado se freqestim == 2)
    sigmaA=8/(winSize-1) # Sigma para filtros de quadratura derivada de Gauss (aplicado se freqestim == 3)

    convmode='valid' # Calcule as respostas do descritor apenas na parte que tem vizinhança completa. Use 'same' se todos os pixels forem incluídos (extrapola np.image com zeros).

    img=np.float64(img) # Converter np.image em double
    r=(winSize-1)/2 # Obtenha o raio do tamanho da janela
    x=np.arange(-r,r+1)[np.newaxis] # Formar coordenadas espaciais na janela

    if freqestim==1:  #  Janela uniforme STFT
        # Filtros STFT básicos
        w0=np.ones_like(x)
        w1=np.exp(-2*np.pi*x*STFTalpha*1j)
        w2=np.conj(w1)

    ## Execute filtros para calcular a resposta de frequência nos quatro pontos. Armazene as partes np.real e np.imaginary separadamente
     # Execute o primeiro filtro
    filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
    filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
    filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
    filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

   # Inicie a matriz de domínio de frequência para quatro coordenadas de frequência (partes np.real e np.imaginary para cada frequência).
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

   ## Execute a quantização e calcule palavras-código LPQ
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

    ## Mude o formato para uint8 se o código LPQ np.image for necessário como saída
    if mode=='im':
        LPQdesc=np.uint8(LPQdesc)

   ## Histograma se necessário
    if mode=='nh' or mode=='h':
        LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

    ## Normalize o histograma se necessário
    if mode=='nh':
        LPQdesc=LPQdesc/LPQdesc.sum()

    return LPQdesc

## Função que realiza a extrações de características de imagens em um diretório
 - Acões:
   - Recebe o diretório, no qual estão localizadas as imagens, faz a contagem das imagens. 
   - Converte a image para escala de cinza
   - O código assume que as imagens já estão no mesmo tamanho
   - Faz as chamadas das funções dos extratores de caracteristicas 
   - Monitora e mostra o tempo de processamento de cada extrator
   - Retorna um objeto que contem uma lista para cada extrator 

In [5]:
def ExtractFeatureDataset(path):
    images_path = os.listdir(path)
    data = []
    for n, image in enumerate(images_path):
        print('Extraindo: ', image, ' Category:', os.path.basename(os.path.normpath(path)), ' Quantidade: ', n, '/', len(images_path))
        img = cv2.imread(os.path.join(path, image))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.max(2)
        mask = np.ones((img.shape[0], img.shape[1]))
        feature_fos = fos_features(img, mask)[0]
        print('FOS-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        aux_feature = glcm_features(img)
        feature_glcm = np.hstack([aux_feature[0], aux_feature[1]])
        print('GLCM-Extraido, %s Segundos' % round((time.time() - start_time),2) )
        feature_lpq = lpq_features(img, 7)
        print('LPQ-Extraido, %s Segundos' % round((time.time() - start_time),2) )

        imgData = imageData()
        imgData.name = image
        imgData.featureFos = list(feature_fos)
        imgData.featureGlcm = list(feature_glcm)
        imgData.featureLpq = list(feature_lpq)
       
        imgData.category = os.path.basename(os.path.normpath(path))
        data.append(imgData)
    return data

### A cedula abaixo realiza a extração das caracteristicas 
- Entrada
    - Diretórios onde estão localizadas as imagens 
- Saida:
    - Geração de um arquivo em csv, que contem a identificação da imagem, classe e as listas das caracteristicas extraidas 

In [6]:
class imageData(object):
    __slots__ = ['name', 
                'featureFos', 
                'featureGlcm', 
                'featureLpq',
                'category']

img1 = imageData()
img1.name = 'imagem'
#Define o caminho do diretório onde estão localizados as imagens para realizar a extração 
folder_path_covid = "C:/Users/junin/Documents/Testecovid"
folder_path_normais = "C:/Users/junin/Documents/Testenormal"

dataCovid = ExtractFeatureDataset(folder_path_covid)
dataNormal = ExtractFeatureDataset(folder_path_normais)


##Combinando para gerar uma lista só para exportarção em CSV
combineData = np.append(dataCovid, dataNormal)
#combineData2 = np.append(combineData, dataIntersticial)
dataSet = combineData

##Titulo de cada atributo
fieldnames = ['Image', 
              'featureFos', 
              'featureGlcm', 
              'featureLpq',
              'Category']

##Função que exporta a lista de extrações em CSV 
def WriteCSVFile(path, fieldnames, dataset):
    file = open(path, 'w', newline='', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(fieldnames)
    for data in dataset:
        objImg = [data.name, 
                  data.featureFos, 
                  data.featureGlcm, 
                  data.featureLpq, 
                  data.category]
        writer.writerow(objImg)

WriteCSVFile('C:/Users/junin/Documents/Git SCC5836/SCC5836-Visualiza-o-Computacional/Kdataset_texture_features_raiox1.csv',fieldnames, dataSet)

Extraindo:  Fig -1 -Backes (1).png  Category: Testecovid  Quantidade:  0 / 1
FOS-Extraido, 0.54 Segundos
GLCM-Extraido, 0.59 Segundos
LPQ-Extraido, 0.76 Segundos
Extraindo:  Fig -1 -Backes (1).png  Category: Testenormal  Quantidade:  0 / 1
FOS-Extraido, 0.81 Segundos
GLCM-Extraido, 0.85 Segundos
LPQ-Extraido, 1.04 Segundos


   **Formato do CSV:**

|Índice|Imagem|featureFos|featureGlcm|featureGlcm|Category|
|:--:|:--:|:--:|:--:|:--:|:--:|
|0|nomeImagem.png|list [...]|list [...]|list [...]|Class|

  **Dimensionalidade dos dados:**

*  dataset_raiox_features_adjusted.csv

|featureFos|featureGlcm|featureGlcm|
|:--:|:--:|:--:|
|512|13|108|

*  dataset_texture_features_raiox.csv
*  kaggle_dataset_texture_features_raiox.csv



* Inicialmente, os valores estavam sendo considerados como uma string. Portanto foi necessario remover os colchetes com o [1:-1] e fazer o split dos dados
* Retorna os dados extraidos em formato de lista

In [7]:
def split_data(input, feature_name, size):
    image_ids = []
    image_features = []
    image_categories = []
    for i in range(size):
        
        x = [ float(v) for v in input.iloc[i][feature_name][1:-1].split(", ") ] 
        x = np.array(x)
        image_features.append(x)
        image_ids.append(input.iloc[i].Image)
        image_categories.append(input.iloc[i].Category)
    return image_ids, image_features, image_categories

In [10]:
#Leitura do Dataset
input = pd.read_csv("Kdataset_texture_features_raiox1.csv")

In [11]:
#Classes no dataset
print("\nDataset completo:")
print(input["Category"].value_counts())


Dataset completo:
Testecovid     1
Testenormal    1
Name: Category, dtype: int64


In [12]:
#print("\nDataset com 2 classes:")
print(input["Category"].value_counts())

Testecovid     1
Testenormal    1
Name: Category, dtype: int64


In [13]:
#Atualizando indices (Removendo os 'furos' dos drops)
input = input.reset_index()
input = input.drop('index', axis=1)
input

,Image,featureFos,featureGlcm,featureLpq,Category
0,Fig -1 -Backes (1).png,"[230.3243073350637, 3719.7131333804423, 255.0,...","[0.5483556439064575, 4231.214005920127, 0.4303...","[0.0010137724041040486, 0.004903891183106987, ...",Testecovid
1,Fig -1 -Backes (1).png,"[230.3243073350637, 3719.7131333804423, 255.0,...","[0.5483556439064575, 4231.214005920127, 0.4303...","[0.0010137724041040486, 0.004903891183106987, ...",Testenormal


In [15]:
#Definindo as features e as funcoes de distancia
feature_names = ["featureFos", "featureGlcm","featureLpq"] #[f for f in input.columns[1:-1]]
#distance_functions = ["euclidean", "cosine", "manhattan"] #Mesmas distancias: l1=manhattan=cityblock  e  l2=euclidean
seed = 42 #Semente de entrada para execucao das Tecnicas de Projecao Multidimensional
#distance_matrix_output = True #Ativa/Desativa a geracao dos arquivos contendo matrizes de dissimilaridade

#Numero de instancias do dataset
size = input.shape[0]
print("Total number of images: ", size)

print("\nDimensionality: ")
#Obtendo as features
image_ids = {}; image_features = {}; image_categories  = {};
for feature_name in feature_names:
    ids, features, categories = split_data(input, feature_name, size)
    print(feature_name, len(features[0]))
    image_ids[feature_name] = ids
    image_features[feature_name] = features
    image_categories[feature_name] = categories

Total number of images:  2

Dimensionality: 
featureFos 16
featureGlcm 28
featureLpq 255
